In [29]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()
client.completions
def perform_task(context, task, tools, tool_choice):
    """Performs a task given the context"""

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": f"Context:\n{context}"
        },
        {
            "role": "user",
            "content": f"Task:\n{task}"
        },
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools= tools,
        tool_choice=tool_choice
    )

    return response


In [4]:
context = "Hello, I am running late and I cannot be there on time for my 1:30 reservation. Would it be possible to reschedule it for 2:30. Also we have one person who cannot make it, so we will only need a table for 5 now. Thank you, - Jane Doe"
tasks =[
    "Classify the email into one of 3 categories, categories=['making a new reservation', 'cancel existing reservation', 'modify existing reservation'] and then extract the values using the appropriate function",
    "given the output, call the corresponding given function name"
]

In [6]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": f"Context:\n{context}"
    },
    {
        "role": "user",
        "content": f"Task:\n{tasks[0]}"
    }
]

In [30]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "perform_action",
            "description": "Take an email text and cataegorize it, then call the actions corresponding tool",
            "parameters": {
                "type": "object",
                "properties": {
                    "action": {
                        "type": "string",
                        "enum": ["new_reservation", "cancel_reservation", "modify_reservation"],
                        "description": "Possible actions to perform on the reservation",
                    },
                },
                "required": ["action"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "modify_reservation",
            "description": "Extract Key/Value pairs for a modify reservation email",
            "parameters": {
                "type": "object",
                "properties": {
                    "client_name": {
                        "type": "string",
                        "description": "Full name of email sender"
                    },
                    "new_party_size": {
                        "type": "string",
                        "description": "The new size of the reservation list"
                    },
                    "new_time": {
                        "type": "string",
                        "description": "The new time for the reservation"
                    },
                },
                "required": ["client_name", "new_party_size", "new_time"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_reservation",
            "description": "Extract Key/Value pairs for a cancel reservation email",
            "parameters": {
                "type": "object",
                "properties": {
                    "client_name": {
                        "type": "string",
                        "description": "Full name of email sender"
                    },
                }
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "new_reservation",
            "description": "Extract Key/Value pairs for a create reservation email",
            "parameters": {
                "type": "object",
                "properties": {
                    "client_name": {
                        "type": "string",
                        "description": "Full name of email sender"
                    },
                    "party_size": {
                        "type": "string",
                        "description": "The new size of the reservation list"
                    },
                    "time": {
                        "type": "string",
                        "description": "The new time for the reservation"
                    },
                }
            }
        }
    }
]

In [24]:
print(messages)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Context:\nHello, I am running late and I cannot be there on time for my 1:30 reservation. Would it be possible to reschedule it for 2:30. Also we have one person who cannot make it, so we will only need a table for 5 now. Thank you, - Jane Doe'}, {'role': 'user', 'content': "Task:\nClassify the email into one of 3 categories, categories=['making a new reservation', 'cancel existing reservation', 'modify existing reservation'] and respond with the string value of the correct category"}, {'role': 'user', 'content': 'Task:\ngiven the output, call the corresponding given function name'}]


In [31]:
res = perform_task(
    context=context,
    task=tasks[0],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "modify_reservation"}}
)
res_action = eval(res.choices[0].message.tool_calls[0].function.arguments).get('action')

In [43]:
for i in range(10):
    res = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "perform_action"}}
    )
    print(eval(res.choices[0].message.tool_calls[0].function.arguments).get('action'))

modify
modify
modify
modify
modify
modify
modify
modify
modify
modify


In [32]:
res.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_YYmDLLkwNsAdJzKCX8pGZqTI', function=Function(arguments='{"client_name":"Jane Doe","new_party_size":"5","new_time":"2:30"}', name='modify_reservation'), type='function')]))

In [27]:
res.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_5WnOv9EEjBX5hWKcLqzRDp2h', function=Function(arguments='{"action":"modify_reservation"}', name='perform_action'), type='function')]

In [28]:
eval(res.choices[0].message.tool_calls[0].function.arguments).get('action')

'modify_reservation'

In [14]:
context = "Hello, I am running late and I cannot be there on time for my 1:30 reservation. Would it be possible to reschedule it for 2:30. Also we have one person who cannot make it, so we will only need a table for 5 now. Thank you, - Jane Doe"
task = "Classify the email into one of 3 categories, categories=['making a new reservation', 'cancel existing reservation', 'modify existing reservation'] and respond with the string value of the correct category"
for i in range(10):
    print(perform_task(context=context, task=task))

modify existing reservation
modify existing reservation
modify existing reservation
modify existing reservation
modify existing reservation
modify existing reservation
'modify existing reservation'
modify existing reservation
modify existing reservation
modify existing reservation


In [10]:
context = "Hello, I am running late and I cannot be there on time for my 1:30 reservation. Would it be possible to reschedule it for 2:30. Also we have one person who cannot make it, so we will only need a table for 5 now. Thank you, - Jane Doe"
task = "Classify the email into one of 3 categories, categories=['making a new reservation', 'cancel existing reservation', 'modify existing reservation'] and respond with the array index of the correct category"
for i in range(10):
    print(perform_task(context=context, task=task))

Category: modify existing reservation
Array index: 2
The email falls under the category of 'modify existing reservation' since Jane Doe is requesting a reschedule of the reservation and a change in the number of guests. The index of this category in the given list is 2.
The email falls under the category of 'modify existing reservation' based on the request to reschedule the reservation from 1:30 to 2:30 and change the number of people from 6 to 5. The array index for this category is 2.
Based on the content of the email, the classification would fall under the 'modify existing reservation' category. The email is requesting a change in the reservation time and a modification to the number of people in the party. Therefore, the array index corresponding to the 'modify existing reservation' category is 2.
The email falls under the category of "modify existing reservation" as the sender is requesting to reschedule the reservation time and reduce the number of guests. The array index for t